In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import json
import os
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import display, Latex

import wandb
api = wandb.Api()

tqdm.pandas()

In [2]:
setting_columns = [
    # Overall
    "tag",
    # "seed",
    "_timestamp",

    # Interventions
    "interventions", "edit", "unlearn", "compression", "model_name",

    # Editing
    "edit_set", 
    "edit_dataset", "number_of_edits",

    # Unlearning
    "rmu_layer_id",

    # Compression
    "wbits", "compression_dataset", "sparsity_ratio",
]
evaluation_columns = [
    "qa_question_count_limit",  # An artifical max number of questions to ask during evaluation. Should be none when not debugging.
    "mmlu accuracy",            # The accuracy of the model on the MMLU dataset. This measures overall model utility. Llama-3 should be ~62%
    "wmdp_bio accuracy",        # The accuracy of the model on the WMDP bio split. This is the unlearning target. Should be ~25% when RMU is applied.
    "wmdp_cyber accuracy",      # The accuracy of the model on the WMDP cyber split. This is the unlearning target. Should be ~25% when RMU is applied.
    "PPL",                      # TODO:
    "PPL edits",                # Perplexity for the edits. Should be low when editing is applied.
    "PPl QA",                   # Perplexity for the QA. Should be low when QA is applied.
    "Generalization",           # TODO: 
    "FLOPs",                    # TODO: 
    "Success recall",           # TODO:
    "Generalization recall",    # TODO:
    "Locality",                 # TODO:
    "Average bits",             # TODO:
    "Rewrite accuracy",         # TODO:
    "PPl edits unmasked",       # TODO:
    "Local recall",             # TODO:
    "Latency",                  # TODO:
]
relevant_columns = setting_columns + evaluation_columns

In [ ]:
# Composable_Interventions has all the results
project_paths = ["dri-ice/Composable_Interventions",]

filter_dict = { "state": "finished" }
data_frames = []
for project_path in project_paths:
    runs = api.runs(project_path, filters=filter_dict)
    
    # Iterate over eachrun and capture the c        onfig and summary metrics
    for run in tqdm(runs, desc=project_path):
        try:
            run_start_datetime = datetime.fromtimestamp(run.summary_metrics["_timestamp"])
            start_cutoff = datetime.strptime("2024-05-18 00:00:00", "%Y-%m-%d %H:%M:%S")
            end_cutoff = datetime.strptime("2024-09-19 00:00:00", "%Y-%m-%d %H:%M:%S")
            if run_start_datetime < start_cutoff or run_start_datetime > end_cutoff:
                continue

            skip_tags = ["test", "hparam_search"]
            should_skip = False
            for tag in skip_tags:
                if tag in run.config["tag"].lower():
                    should_skip = True
            
            if should_skip:
                continue

            config_frame = pd.DataFrame([run.config])
            summary_frame = pd.DataFrame([run.summary_metrics])
            combined_frame = pd.concat([config_frame, summary_frame], axis=1)
            data_frames.append(combined_frame)
        except Exception as e:
            continue
            # print(f"Error processing run {run.id}: {e}")


In [12]:
unlearning_interventions = ["rmu"]
editing_interventions = ["lora"]
pruning_interventions = ["wanda"]
pruning_levels = [0.25, 0.35, 0.45, 0.55, 0.65, 0.75]
quant_interventions = ["awq"]
quant_levels = [2, 3, 4, 5, 6, 8]

experiment_combinations = []

# Unlearning and Editing
for unlearner in unlearning_interventions:
    for editor in editing_interventions:
        experiment_combinations.append({
            "interventions": ["unlearn", "edit"],
            "edit": editor,
            "unlearn": unlearner,
            "wbits": None,
            "sparsity_ratio": None,
        })
        experiment_combinations.append({
            "interventions": ["edit", "unlearn"],
            "edit": editor,
            "unlearn": unlearner,
            "wbits": None,
            "sparsity_ratio": None,
        })

# Unlearning and Compression
for unlearner in unlearning_interventions:
    for pruner in pruning_interventions:
        for pruning_level in pruning_levels:
            experiment_combinations.append({
                "interventions": ["unlearn", "compression"],
                "edit": None,
                "unlearn": unlearner,
                "compression": pruner,
                "sparsity_ratio": pruning_level,
            })
            experiment_combinations.append({
                "interventions": ["compression", "unlearn"],
                "edit": None,
                "unlearn": unlearner,
                "compression": pruner,
                "sparsity_ratio": pruning_level,
            })

    for quantizer in quant_interventions:
        for quant_level in quant_levels:
            experiment_combinations.append({
                "interventions": ["unlearn", "compression"],
                "edit": None,
                "unlearn": unlearner,
                "compression": quantizer,
                "wbits": quant_level,
                "sparsity_ratio": None,
            })
            experiment_combinations.append({
                "interventions": ["compression", "unlearn"],
                "edit": None,
                "unlearn": unlearner,
                "compression": quantizer,
                "wbits": quant_level,
                "sparsity_ratio": None,
            })

# Editing and Compression
for editor in editing_interventions:
    for pruner in pruning_interventions:
        for pruning_level in pruning_levels:
            experiment_combinations.append({
                "interventions": ["edit", "compression"],
                "edit": editor,
                "unlearn": None,
                "compression": pruner,
                "sparsity_ratio": pruning_level,
            })
            experiment_combinations.append({
                "interventions": ["compression", "edit"],
                "edit": editor,
                "unlearn": None,
                "compression": pruner,
                "sparsity_ratio": pruning_level,
            })

    for quantizer in quant_interventions:
        for quant_level in quant_levels:
            experiment_combinations.append({
                "interventions": ["edit", "compression"],
                "edit": editor,
                "unlearn": None,
                "compression": quantizer,
                "wbits": quant_level,
                "sparsity_ratio": None,
            })
            experiment_combinations.append({
                "interventions": ["compression", "edit"],
                "edit": editor,
                "unlearn": None,
                "compression": quantizer,
                "wbits": quant_level,
                "sparsity_ratio": None,
            })

# No interventions
experiment_combinations.append({
    "interventions": [],
    "edit": None,
    "unlearn": None,
    "compression": None,
    "wbits": None,
    "sparsity_ratio": None,
})

print(f"Total experiment combinations for Mistral ablation: {len(experiment_combinations)}")
for experiment in experiment_combinations:
    
        


Total experiment combinations for Mistral ablation: 51
